In [ ]:
!pip install transformers

In [4]:
# Instalar librerías
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd


# Preprocesamiento y limpieza de la BD.



In [5]:
# Ficheros de entrada
ED_TRAIN1 = "data/ED_train1.csv"
ED_TRAIN2 = "data/ED_train2.csv"
ED_VALID = "data/ED_valid.csv"
ED_TEST = "data/ED_test.csv"

# Ficheros procesados
ED_TRAIN_P1 = "data/ED_train_p1.csv"
ED_TRAIN_P2 = "data/ED_train_p2.csv"
ED_VALID_P = "data/ED_valid_p.csv"
ED_TEST_P = "data/ED_test_p.csv"

In [6]:
def BD_preprocessing(csv_in, csv_out, s):
    '''
    Procesa los ficheros .csv correspondientes a la BD EMPATHETICDIALOGUES que servirán como 
    entrada para el entrenamiento del codificador BERT
    
    Parámetros:
        csv_in: Nombre del fichero de entrada
        csv_out: Nombre del fichero de salida
        s: separador en los ficheros .csv
    '''
    df = pd.read_csv(csv_in,sep=s) 
    
    len_max = 0
    # Pone en una misma fila, las parejas de turnos de conversación
    df['utterance2'] = ""
    conv = df['conv_id'][0]
    for index, row in df.iterrows(): 
        if (index > 0) and (conv == row['conv_id']): # Para todas las lineas de la misma conversación.
            df["utterance2"][index-1] = row["utterance"]
        else:
            conv = row['conv_id'] # Cambio al siguiente diálogo.

    # Borra las lineas huérfanas. Corresponden al último turno de cada conversación.
    df = df.drop(df[df['utterance2']==""].index)
    
   
    # Corregimos sustituyendo "_comma_" por ","
    df['utterance']= df['utterance'].str.replace("_comma_", ",", case = False) 
    df['utterance2']= df['utterance2'].str.replace("_comma_", ",", case = False)  
    
    # Asignación de la clasificación según la emoción
    df['emotion'] = df['context']
    
    # Listas de emociones según nivel de recompensa
    nivel1 = ['anxious','devastated','disappointed','disgusted','furious','jealous','lonely','terrified'] # Recompensa -2
    nivel2 = ['afraid','angry','annoyed','apprehensive','ashamed','embarrassed','guilty','sad'] # Recompensa -1
    nivel3 = ['caring','confident','content','joyful','nostalgic','prepared','sentimental','trusting'] # Recompensa 1
    nivel4 = ['anticipating','excited','faithful','grateful','hopeful','impressed','proud','surprised'] # Recompensa 2

    # Sustituir la emoción con el nivel que le corresponde
    # En primer lugar informé con 1,2,3,4
    df['emotion'] = df['emotion'].replace(nivel1, 0)
    df['emotion'] = df['emotion'].replace(nivel2, 1)
    df['emotion'] = df['emotion'].replace(nivel3, 2)
    df['emotion'] = df['emotion'].replace(nivel4, 3)
    
    # Eliminar las columnas que no necesitamos
    df = df.drop(['conv_id','utterance_idx','context','prompt','speaker_idx','selfeval','tags','head9','head10','head11','head12'],axis=1)
    
    # Guardar csv
    df.to_csv(csv_out,index=False,sep=';')
    
    

In [ ]:
BD_preprocessing(ED_TRAIN1,ED_TRAIN_P1,';')
BD_preprocessing(ED_TRAIN2,ED_TRAIN_P2,';')
BD_preprocessing(ED_VALID,ED_VALID_P,',')
BD_preprocessing(ED_TEST,ED_TEST_P,',')

# 4. Entrenamiento aprendizaje reforzado con REINFORCE